# 42. MNIST CNN with Dropout

<p style="text-align: right;">
blackdew7@gmail.com<br>
Your name :
</p>

### 텐서플로 실습시간 그 여섯번째!<br>

#### 선행지식
1. TensorFlow 다루기 기초
2. 모델링을 한다는 것에 대한 이해.
3. Supervised Learning 중 Classification에 대한 기본 지식.
4. CNN에 대한 기본 구조와 개념

#### 실습목표
1. CNN의 구조를 Graph로 그려낼 수 있다.
2. 그려낸 Graph를 텐서플로우를 이용해 코딩할 수 있다.
3. Dropout을 이해하고 사용할 수 있다.

#### 사용데이터.

01. Multinomial Classification : http://yann.lecun.com/exdb/mnist/

## 00. 라이브러리 불러오기

In [ ]:
# 메모리 초기화
%reset

# MNIST and Convolutional Neural Network
import tensorflow as tf
import random

## 01. 데이터 불러오기 & 전처리

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./data/MNIST_data/", one_hot=True)

## 02. Prepare Variables for a CNN Graph with TF

In [ ]:
'''
지난 시간에 배운 Prepare Variables 코드를 완성해보자.

여러분이 직접 스크립트를 짜야 합니다.
복사하여 붙여넣기를 하지 마시고, 직접 타이핑 하는 것을 권장합니다.
'''

# Graph Clear
tf.reset_default_graph()
tf.set_random_seed(2017) # random seeding - reproduct

############################
# Place Holders

X = tf.placeholder(tf.float32, [None, 784], name="X")
X_img = tf.reshape(X, [-1, 28, 28, 1], name="X_img")
Y = tf.placeholder(tf.float32, [None, 10], name="Y")


## 03. Make a CNN Graph with TF

### 03.1 First HIdden Layer

In [ ]:
'''
지난 시간에 배운 First Hidden Layer 코드를 완성해보자

__ <= 이렇게 된 빈 칸을 채워주시면 됩니다. 
'''

# for Conv Layer 01 filter - shape=(3, 3, 1)
W1 = tf.Variable(tf.random_normal([3, 3, 1, 64], stddev=0.01), name="W1")

# Convolution Layer 01 -> (?, 28, 28, 32)
L1_conv = tf.nn.conv2d(X_img, W1, strides=[1, 1, 1, 1], padding='SAME', name="L1_conv")
L1_conv = tf.nn.relu(L1_conv, name="L1_relu")

# Pooling Layer 01 -> (?, 14, 14, 32)
L1_pool = tf.nn.max_pool(L1_conv, ksize=[1, 2, 2, 1], 
                         strides=[1, 2, 2, 1], padding='SAME', name="L1_pool")

### 03.2 Second Hidden Layer

In [ ]:
'''
지난 시간에 배운 Second Hidden Layer 코드를 완성해보자

__ <= 이렇게 된 빈 칸을 채워주시면 됩니다. 
'''

# for Conv Layer 02 filter - shape=(3, 3, 32)
W2 = tf.Variable(tf.random_normal([3, 3, 64, 128], stddev=0.01), name="W2")

# Convolution Layer 02 -> (?, 14, 14, 32)
L2_conv = tf.nn.conv2d(L1_pool, W2, strides=[1, 1, 1, 1], padding='SAME', name="L2_conv")
L2_conv = tf.nn.relu(L2_conv, name="L2_relu")

# Pooling Layer 02 -> (?, 7, 7, 64)
L2_pool = tf.nn.max_pool(L2_conv, ksize=[1, 2, 2, 1], 
                         strides=[1, 2, 2, 1], padding='SAME', name="L2_pool")

### 03.3 Fully Connected Layer

#### 지난 시간에 배운 fc layer

In [ ]:
'''
지난 시간에 배운 Fully Connected Layer 코드를 완성해보자

__ <= 이렇게 된 빈 칸을 채워주시면 됩니다. 
'''

###############################
# flatten layer
L_flat = tf.reshape(L2_pool, [-1, 7 * 7 * 128], name="L_flat")


###############################
# fully connected layer

# for Final FC 7x7x64 inputs -> 10 outputs
W_fc = tf.Variable(tf.random_normal([7 * 7 * 128, 10], stddev=0.01), name="W_fc")

# bias
b = tf.Variable(tf.random_normal([10]), name="b")

# fc layer
logits = tf.add(tf.matmul(L_flat, W_fc), b, name='logits')

#### flattern layer

In [ ]:
# flatten layer
L_flat = tf.reshape(L2_pool, [-1, 7 * 7 * 128], name="L_flat")

#### first FC layer

In [ ]:
# for Final FC 7x7x64 inputs -> 1000
W1_fc = tf.Variable(tf.random_normal([7 * 7 * 128, 1000], stddev=0.01), name="W1_fc")

# for Final FC Layer 1: 7x7x64 inputs -> 1000
L1_fc = tf.matmul(L_flat, W1_fc, name="L1_fc")
L1_fc = tf.nn.relu(L1_fc, name="L1_fc_relu")

#### Dropout

![image](https://2.bp.blogspot.com/-WXlVLu2mT4g/WGUcrNdmzcI/AAAAAAAALHA/LmUZbEsJHrw4EjpIkGDVgPzZte4rcM8bwCLcB/s1600/dropout.png)

In [ ]:
# dropout 비율 결정
keep_prob = tf.placeholder(tf.float32, name="keep_prob")
L1_fc_dropout = tf.nn.dropout(L1_fc, keep_prob=keep_prob, name="dropout")

#### second FC layer

In [ ]:
# for Final FC 1000 inputs -> 10
W2_fc = tf.Variable(tf.random_normal([1000, 10], stddev=0.01), name="W2_fc")

# bias
b = tf.Variable(tf.random_normal([10]), name="bias")

# for Final FC Layer 2: 1000 inputs -> 10
logits = tf.add(tf.matmul(L1_fc_dropout, W2_fc), b, name="logits")

### 03.4 Cost & Optimizer

In [ ]:
'''
지난 시간에 배운 Cost & Optimizer 코드를 완성해보자

__ <= 이렇게 된 빈 칸을 채워주시면 됩니다. 
'''

learning_rate = 0.001

# Cost(loss) function & Optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

### 03.6 실습 - Make a CNN Graph with TF

In [ ]:
'''
CNN Graph 코드를 완성해보자

여러분이 직접 스크립트를 짜야 합니다.
복사하여 붙여넣기를 하지 마시고, 직접 타이핑 하는 것을 권장합니다.
'''

#########################
# ConvLayer 01 

with tf.name_scope("layer1_conv"):

    # for Conv Layer 01 filter - shape=(3, 3, 1)
    W1 = 

    # Convolution Layer 01 -> (?, 28, 28, 32)
    L1_conv = 
    L1_conv = 
    
    # Pooling Layer 01 -> (?, 14, 14, 32)
    L1_pool = 

    
#########################
# ConvLayer 02

with tf.name_scope("layer2_conv"):

    # for Conv Layer 02 filter - shape=(3, 3, 32)
    W2 = 

    # Convolution Layer 02 -> (?, 14, 14, 32)
    L2_conv = 
    L2_conv = 

    # Pooling Layer 02 -> (?, 7, 7, 64)
    L2_pool = 

    
#########################
# Fully Connected Layer 01

with tf.name_scope("layer1_fc"):

    ###############################
    # Flatten layer
    L_flat = 

    # for Final FC 7x7x64 inputs -> 1000
    W1_fc = tf.Variable(tf.random_normal([7 * 7 * 64, 1000], stddev=0.01), name="W1_fc")

    # for Final FC Layer 1: 7x7x64 inputs -> 1000
    L1_fc = tf.matmul(L_flat, W1_fc, name="L1_fc")
    L1_fc = tf.nn.relu(L1_fc, name="L1_fc_relu")

    #########################
    # Dropout
    keep_prob = tf.placeholder(tf.float32, name="keep_prob")
    L1_fc_dropout = tf.nn.dropout(L1_fc, keep_prob=keep_prob, name="dropout")
    #########################
    
#########################
# Fully Connected Layer 02

with tf.name_scope("layer2_fc"):

    # for Final FC 1000 inputs -> 10
    W2_fc = tf.Variable(tf.random_normal([1000, 10], stddev=0.01), name="W2_fc")

    # bias
    b = tf.Variable(tf.random_normal([10]), name="bias")

    # for Final FC Layer 2: 1000 inputs -> 10
    logits = tf.add(tf.matmul(L1_fc_dropout, W2_fc), b, name="logits")

    
#########################
# Cost & Optimizer

learning_rate = 0.001

with tf.name_scope("Optimizer"):
    # Cost(loss) function & Optimizer
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y), name="cost")
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)


## 04. Traning & Evaluation

In [ ]:
'''
지난 시간에 배운 Traning & Evaluation 코드를 완성해보자

여러분이 직접 스크립트를 짜야 합니다.
복사하여 붙여넣기를 하지 마시고, 직접 타이핑 하는 것을 권장합니다.
'''

###########################
# Initialize
sess = 
sess.run(tf.______())

###########################
# Training

print('Learning started. It takes sometime.')

epochs = 2
batch_size = 500

for epoch in range(epochs):
    print("%dth epoch" % (epoch + 1))

    # 총 갯수: 55000개를 500개씩 (batch_size) 나누어 훈련 
    #        => 110번 (n_of_batches) 훈련하게 된다. 
    n_of_batches = int(mnist.train.num_examples / batch_size)
    
    for i in range(n_of_batches):
        X_batch, Y_batch = mnist.train.next_batch(batch_size)
        # 학습 진행
        sess.run(optimizer, feed_dict={______})
        
        if ((i + 1) % 2 == 0):
            # 학습 상황 디스플레이
            loss = sess.run(cost, feed_dict={______})
            print("%dth records, training cost: %.3f" % (((i + 1) * batch_size), loss))
            
###########################
# Evaluation

# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
accuracy = sess.run(acc, feed_dict={______})

print('Accuracy: %.2f' % (acc * 100))

## 05. 실습 해답

### 05.1 Import Library & Prepare MNIST Data

In [ ]:
import tensorflow as tf
from tbij import show_graph # Tensor Board In Jupyter notebook. You can see this later.

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./data/MNIST_data/", one_hot=True)


### 05.2 Prepare Variables for a CNN Graph with TF

In [ ]:
# Graph Clear
tf.reset_default_graph()
tf.set_random_seed(2017) # random seeding - reproduct

############################
# Place Holders

X = tf.placeholder(tf.float32, [None, 784], name="X")
X_img = tf.reshape(X, [-1, 28, 28, 1], name="X_img")
Y = tf.placeholder(tf.float32, [None, 10], name="Y")


### 05.3 Make a CNN Graph & Run Session with TF

In [ ]:
#########################
# ConvLayer 01 

with tf.name_scope("layer1_conv"):

    # for Conv Layer 01 filter - shape=(3, 3, 1)
    W1 = tf.Variable(tf.random_normal([3, 3, 1, 64], stddev=0.01), name="W1")

    # Convolution Layer 01 -> (?, 28, 28, 32)
    L1_conv = tf.nn.conv2d(X_img, W1, strides=[1, 1, 1, 1], padding='SAME', name="L1_conv")
    L1_conv = tf.nn.relu(L1_conv, name="L1_relu")

    # Pooling Layer 01 -> (?, 14, 14, 32)
    L1_pool = tf.nn.max_pool(L1_conv, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name="L1_max_pool")

    
#########################
# ConvLayer 02

with tf.name_scope("layer2_conv"):

    # for Conv Layer 02 filter - shape=(3, 3, 32)
    W2 = tf.Variable(tf.random_normal([3, 3, 64, 128], stddev=0.01), name="W2")

    # Convolution Layer 02 -> (?, 14, 14, 32)
    L2_conv = tf.nn.conv2d(L1_pool, W2, strides=[1, 1, 1, 1], padding='SAME', name="L2_conv")
    L2_conv = tf.nn.relu(L2_conv, name="L2_relu")

    # Pooling Layer 02 -> (?, 7, 7, 64)
    L2_pool = tf.nn.max_pool(L2_conv, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name="L2_max_pool")

    
#########################
# Fully Connected Layer 01

with tf.name_scope("layer1_fc"):

    ###############################
    # Flatten layer
    L_flat = tf.reshape(L2_pool, [-1, 7 * 7 * 128], name="L_flat")

    # for Final FC 7x7x64 inputs -> 1000
    W1_fc = tf.Variable(tf.random_normal([7 * 7 * 128, 1000], stddev=0.01), name="W1_fc")

    # for Final FC Layer 1: 7x7x64 inputs -> 1000
    L1_fc = tf.matmul(L_flat, W1_fc, name="L1_fc")
    L1_fc = tf.nn.relu(L1_fc, name="L1_fc_relu")

    #########################
    # Dropout
    keep_prob = tf.placeholder(tf.float32, name="keep_prob")
    L1_fc_dropout = tf.nn.dropout(L1_fc, keep_prob=keep_prob, name="dropout")
    #########################
    
    
#########################
# Fully Connected Layer 02

with tf.name_scope("layer2_fc"):

    # for Final FC 1000 inputs -> 10
    W2_fc = tf.Variable(tf.random_normal([1000, 10], stddev=0.01), name="W2_fc")

    # bias
    b = tf.Variable(tf.random_normal([10]), name="bias")

    # for Final FC Layer 2: 1000 inputs -> 10
    logits = tf.add(tf.matmul(L1_fc_dropout, W2_fc), b, name="logits")

    
#########################
# Cost & Optimizer

learning_rate = 0.001

with tf.name_scope("Optimizer"):
    # Cost(loss) function & Optimizer
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y, name="cross_entropy")
    cost = tf.reduce_mean(cross_entropy, name="cost")
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)


### 05.4 Train & Evaluation

In [ ]:
#########################
# Session initialize

sess = tf.Session()
sess.run(tf.global_variables_initializer())

###########################
# Traning

print('Learning started. It takes sometime.')

epochs = 2
batch_size = 500

for epoch in range(epochs):
    print("%dth epoch" % (epoch + 1))

    # 총 갯수: 55000개를 500개씩 (batch_size) 나누어 훈련 
    #        => 110번 (n_of_batches) 훈련하게 된다. 
    n_of_batches = int(mnist.train.num_examples / batch_size)
    
    for i in range(n_of_batches):
        X_batch, Y_batch = mnist.train.next_batch(batch_size)
        # 학습 진행
        sess.run(optimizer, feed_dict={X: X_batch, Y: Y_batch, keep_prob: 0.7})
        
        if ((i + 1) % 2 == 0):
            # 학습 상황 디스플레이
            loss = sess.run(cost, feed_dict={X: X_batch, Y: Y_batch, keep_prob: 0.7})
            print("%dth records, training cost: %.3f" % (((i + 1) * batch_size), loss))
            
print("Training Complete")


###########################
# Evaluation
with tf.name_scope("Prediction"):
    # Test model and check accuracy
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1), name="compare")
    acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name="accuracy")
    accuracy = sess.run(acc, feed_dict={X: mnist.test.images, Y: mnist.test.labels, keep_prob: 1.0})
    
print('Accuracy: %.2f' % (accuracy * 100))

#### 그래프도 그려보자

In [ ]:
from tbij import show_graph # Tensor Board In Jupyter notebook. You can see this later.
show_graph(tf.get_default_graph().as_graph_def())

### 성능을 한 번 확인해 보자. 제대로 되고 있는걸까?

In [ ]:
# Get one and predict
r = random.randint(0, mnist.test.num_examples - 1)
print("Label: ", sess.run(tf.argmax(mnist.test.labels[r:r + 1], 1)))
print("Prediction: ", sess.run(tf.argmax(logits, 1), 
                               feed_dict={X: mnist.test.images[r:r + 1],
                                         keep_prob: 1.0}))

import matplotlib.pyplot as plt
plt.imshow(mnist.test.images[r:r + 1].reshape(28, 28), cmap='Greys', interpolation='nearest')
plt.show()

In [ ]:
# 세션을 종료하자
sess.close()

## 06. 대체 뭘 학습한 것인가? 

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import math
import numpy as np

def plot_conv_weights(weights, input_channel=0):
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        w = sess.run(weights)

        w_min = np.min(w)
        w_max = np.max(w)

        num_filters = w.shape[3]
        num_grids = math.ceil(math.sqrt(num_filters))
        fig, axes = plt.subplots(num_grids, num_grids)

        for i, ax in enumerate(axes.flat):
            if i < num_filters:
                img = w[:, :, input_channel, i]
                ax.imshow(img, vmin=w_min, vmax=w_max, interpolation='nearest', cmap='seismic')

            ax.set_xticks([])
            ax.set_yticks([])

        plt.show()

In [ ]:
plot_conv_weights(weights=W1)

In [ ]:
plot_conv_weights(weights=W2, input_channel=0)

In [ ]:
plot_conv_weights(weights=W2, input_channel=1)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import math
import numpy as np

def plot_conv_layer(layer, image):

    with tf.Session() as sess:

        sess.run(tf.global_variables_initializer())

        feed_dict = {X: [image]}
        values = sess.run(layer, feed_dict=feed_dict)

        num_filters = values.shape[3]
        num_grids = math.ceil(math.sqrt(num_filters))
        fig, axes = plt.subplots(num_grids, num_grids)

        for i, ax in enumerate(axes.flat):
            if i < num_filters:
                img = values[0, :, :, i]
                ax.imshow(img, interpolation='nearest', cmap='binary')
                
            ax.set_xticks([])
            ax.set_yticks([])

        plt.show()

In [ ]:
img_shape = (28, 28)

def plot_image(image):
    plt.imshow(image.reshape(img_shape),
               interpolation='nearest',
               cmap='binary')

    plt.show()
    
image1 = mnist.test.images[0]
plot_image(image1)

image2 = mnist.test.images[1]
plot_image(image2)

In [ ]:
plot_conv_layer(layer=L1_pool, image=image1)

In [ ]:
plot_conv_layer(layer=L2_pool, image=image2)